In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [2]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import LDA


In [3]:
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [46]:
!pwd

/Users/youssouf/Documents/PythonCOde


In [47]:
df = spark.read.parquet("/Users/youssouf/Downloads/prediction_final_20211006.parquet")

df.show(truncate = True)

+-------------+--------------------+-------------+--------------------+-------------+
|       msisdn|      probability_dt|prediction_dt|      probability_rf|prediction_rf|
+-------------+--------------------+-------------+--------------------+-------------+
|2250700004874|[0.08443540183112...|          1.0|[0.36505015719161...|          1.0|
|2250700012204|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
|2250700012536|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
|2250700014915|[0.08443540183112...|          1.0|[0.36876257793014...|          1.0|
|2250700020810|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
|2250700021781|[0.08443540183112...|          1.0|[0.36876257793014...|          1.0|
|2250700025447|[0.23142144638403...|          1.0|[0.30806991239676...|          1.0|
|2250700025686|[0.08443540183112...|          1.0|[0.33661093239710...|          1.0|
|2250700029582|[0.23142144638403...|          1.0|[0.2

mariam joue a la balle

In [48]:

df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")

df_fraudeur_FRA.show(5)

+--------------+---------+-----+---------+-----------+
|Date_DETECTION|MSISDN_SB|OUTIL|OPERATEUR|DESTINATION|
+--------------+---------+-----+---------+-----------+
|    28/09/2021| 00004846|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00048563|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058736|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058747|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058748|   FR|   ORANGE|     ON NET|
+--------------+---------+-----+---------+-----------+
only showing top 5 rows



In [59]:
def check_fraudeur_date(path_df, date) :
    from datetime import datetime

    #path_df = "/Users/youssouf/Downloads/simbox_v4_prediction_" + model + "_" + date + ".parquet"
    #"/Users/youssouf/Documents/simboxv2/prediction_gbt_vbm_30_v4_dist.parquet"
    df = spark.read.parquet(path_df)
    df1 = df
      
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")
    df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))

    d = datetime.strptime(date , '%Y%m%d')
    d = datetime.strftime(d, "%d/%m/%Y")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("Date_DETECTION") == d)
    
    #df_fraudeur = df_fraudeur.select("msisdn").join(df.select("msisdn"), ["msisdn"], "inner")
    
    a = df1.join(df_fraudeur, ["msisdn"], "inner").count() 
    res = {"NOMBRE DE DETECTION" : df1.count() , 
           "NOMBRE DE FRA" : df_fraudeur.count() ,
           "NOMBRE COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").count(),
           "NUMERO COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").select("msisdn").show(a)
           #, 
           #"NOMBRE FRAU/NON-FRAU": df.groupBy("prediction").count().show() 
          }

    return res

In [17]:
def check_fraudeur_date2(path_df, date) :
    from datetime import datetime

    #path_df = "/Users/youssouf/Downloads/simbox_v4_prediction_" + model + "_" + date + ".parquet"
    #"/Users/youssouf/Documents/simboxv2/prediction_gbt_vbm_30_v4_dist.parquet"
    df = spark.read.parquet(path_df)
    df1 = df.filter(col("prediction") == 1)
      
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")
    df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))

    d = datetime.strptime(date , '%Y%m%d')
    d = datetime.strftime(d, "%d/%m/%Y")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("Date_DETECTION") == d)
    
    df_fraudeur = df_fraudeur.select("msisdn").join(df.select("msisdn"), ["msisdn"], "inner")

    a = df1.join(df_fraudeur, ["msisdn"], "inner").count() 
    
    res = {"NOMBRE DE DETECTION" : df.count() , 
           "NOMBRE DE FRA" : df_fraudeur.count() ,
           "NOMBRE COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").count() ,
           "NUMERO COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").select("msisdn").show(a) ,
           "NOMBRE FRAU/NON-FRAU": df.groupBy("prediction").count().show() 
          }

    return res

In [7]:
def check_fraudeur_date3(df , classe ,date) :
    from datetime import datetime

    #path_df = "/Users/youssouf/Downloads/simbox_v4_prediction_" + model + "_" + date + ".parquet"
    #"/Users/youssouf/Documents/simboxv2/prediction_gbt_vbm_30_v4_dist.parquet"
    #df = spark.read.parquet(path_df)
    df1 = df.filter(col("prediction") == classe)
      
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")
    df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))

    d = datetime.strptime(date , '%Y%m%d')
    d = datetime.strftime(d, "%d/%m/%Y")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("Date_DETECTION") == d)
    
    df_fraudeur = df_fraudeur.select("msisdn").join(df.select("msisdn"), ["msisdn"], "inner")
    
    a = df1.join(df_fraudeur, ["msisdn"], "inner").count() 
    res = {"NOMBRE DE DETECTION" : df.count() , 
           "NOMBRE DE FRA" : df_fraudeur.count() ,
           "NOMBRE COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").count() ,
           "NUMERO COMMUN" : df1.join(df_fraudeur, ["msisdn"], "inner").select("msisdn").show(a) ,
           "NOMBRE FRAU/NON-FRAU": df.groupBy("prediction").count().show() 
          }

    return res

In [48]:
def check_fraudeur_date4(path , date) :
    from datetime import datetime

    #path_df = "/Users/youssouf/Downloads/simbox_v4_prediction_" + model + "_" + date + ".parquet"
    #"/Users/youssouf/Documents/simboxv2/prediction_gbt_vbm_30_v4_dist.parquet"
    #df = spark.read.parquet(path_df)
    df = spark.read.option("header", "true").csv(path)
      
    df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")
    df_fraudeur_FRA = df_fraudeur_FRA.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))

    #d = datetime.strptime(date , '%Y%m%d')
    #d = datetime.strftime(d, "%d/%m/%Y")
    
    df_fraudeur = df_fraudeur_FRA.filter(col("Date_DETECTION") == date)
        
    compa = df.join(df_fraudeur , ["msisdn"], "inner")
    
    a = df.join(df_fraudeur, ["msisdn"], "inner").count() 
    res = {"NOMBRE DE DETECTION" : df.select("msisdn").distinct().count()  , 
           "NOMBRE DE FRA" : df_fraudeur.select("msisdn").distinct().count() ,
           "NOMBRE COMMUN" : compa.select("msisdn").distinct().count() ,
           "NUMERO COMMUN" : compa.select("msisdn").show(a) ,
           "NOMBRE FRAU/NON-FRAU": "" #df.groupBy("prediction").count().show() 
          }

    return res

In [50]:
check_fraudeur_date4("/Users/youssouf/Downloads/prediction_final_kmeans20211007.csv", "07/10/2021")

+-------------+
|       msisdn|
+-------------+
|2250798123777|
|2250787432803|
|2250759853146|
|2250704697493|
|2250704696512|
|2250700606539|
|2250799875286|
|2250748120063|
|2250700606408|
|2250704664182|
|2250704662981|
|2250700606697|
|2250704697499|
|2250789805125|
|2250799875294|
|2250704680460|
|2250700808386|
|2250767661648|
|2250704664332|
|2250704671295|
|2250700606660|
|2250704138218|
|2250704671384|
|2250768562664|
|2250700316662|
|2250700473511|
|2250700692534|
|2250700606144|
|2250799230516|
|2250704663759|
|2250700606924|
|2250768007956|
|2250700691494|
|2250778452888|
|2250700606705|
|2250700695323|
|2250704696221|
|2250799896715|
|2250799896717|
|2250704697500|
|2250759277455|
|2250787432682|
|2250700691438|
|2250700695315|
|2250704696233|
|2250799896532|
|2250700473770|
|2250700606977|
|2250704664167|
|2250704696529|
|2250704664179|
|2250799875538|
|2250700808380|
|2250704664281|
|2250704664177|
|2250704664315|
|2250704664175|
|2250704696527|
|2250704663412|
|2250700

{'NOMBRE DE DETECTION': 675089,
 'NOMBRE DE FRA': 460,
 'NOMBRE COMMUN': 198,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': ''}

In [38]:
df_fraudeur_FRA = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/Détection_SIMBOX1.csv")

df_fraudeur_FRA.filter(col("Date_DETECTION") == "06/10/2021").count()

417

In [42]:
df_fraudeur_FRA.filter(col("Date_DETECTION") == "06/10/2021").select("MSISDN_SB").distinct().count()

414

In [27]:
check_fraudeur_date("/Users/youssouf/Downloads/prediction_final_20211007.parquet", "20211007")

{'NOMBRE DE DETECTION': 848520, 'NOMBRE DE FRA': 462, 'NOMBRE COMMUN': 200}

In [62]:
check_fraudeur_date("/Users/youssouf/Downloads/prediction_final_20211008.parquet", "20211008")

+-------------+
|       msisdn|
+-------------+
|2250799896538|
|2250798949382|
|2250799338161|
|2250799896529|
|2250704697993|
|2250788074827|
|2250704695797|
|2250799896711|
|2250799896726|
|2250700691692|
|2250799575408|
|2250799668446|
|2250704663763|
|2250704675652|
|2250704664392|
|2250799860869|
|2250799896551|
|2250704679668|
|2250704664318|
|2250799875191|
|2250704679675|
|2250704679626|
|2250704655715|
|2250704669873|
|2250704655763|
|2250704679670|
|2250700687132|
|2250704663758|
|2250700316501|
|2250704679671|
|2250700606668|
|2250799870380|
|2250769156402|
|2250700691539|
|2250799896701|
|2250704695807|
|2250799896707|
|2250787967095|
|2250700437234|
|2250704695907|
|2250799966259|
|2250704679674|
|2250704679604|
|2250700437245|
|2250700437245|
|2250797264316|
|2250799896547|
|2250799896690|
|2250704695824|
|2250704663760|
|2250704679624|
|2250767613048|
|2250700691307|
|2250789378392|
|2250799896591|
|2250704138187|
|2250769728792|
|2250700180722|
|2250704679609|
|2250799

{'NOMBRE DE DETECTION': 1341246,
 'NOMBRE DE FRA': 540,
 'NOMBRE COMMUN': 99,
 'NUMERO COMMUN': None}

In [13]:
check_fraudeur_date("/Users/youssouf/Downloads/prediction_final_20211010.parquet", "20211010")

{'NOMBRE DE DETECTION': 943755, 'NOMBRE DE FRA': 127, 'NOMBRE COMMUN': 13}

In [63]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_rf_20211007.parquet", "20211007")

+-------------+
|       msisdn|
+-------------+
|2250700606374|
|2250704663762|
|2250700691295|
|2250787432599|
|2250799896678|
|2250704662994|
|2250799892531|
|2250798123777|
|2250700690230|
|2250700808382|
|2250704697484|
|2250704684335|
|2250704695894|
|2250798952910|
|2250704695806|
|2250704664295|
|2250769524300|
|2250787432803|
|2250704662977|
|2250704663773|
|2250704695116|
|2250799890903|
|2250759853146|
|2250799894474|
|2250700690137|
|2250700822048|
|2250700687066|
|2250704694927|
|2250704697493|
|2250704662115|
|2250704662116|
|2250704662960|
|2250704696505|
|2250704696524|
|2250759859455|
|2250700606431|
|2250704697955|
|2250704694253|
|2250704696257|
|2250799891130|
|2250704663765|
|2250700690140|
|2250704136227|
|2250700606280|
|2250700697976|
|2250704696512|
|2250700691856|
|2250700699613|
|2250700606568|
|2250704663378|
|2250704664316|
|2250779252685|
|2250799875492|
|2250704696540|
|2250704138180|
|2250704664341|
|2250799896693|
|2250700606539|
|2250799875286|
|2250704

{'NOMBRE DE DETECTION': 6095341,
 'NOMBRE DE FRA': 359,
 'NOMBRE COMMUN': 270,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [29]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_rf_20211008.parquet", "20211008")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|4160409|
|       1.0|3816761|
+----------+-------+



{'NOMBRE DE DETECTION': 7977170,
 'NOMBRE DE FRA': 260,
 'NOMBRE COMMUN': 142,
 'NOMBRE FRAU/NON-FRAU': None}

In [18]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211008.parquet", "20211008")

+-------------+
|       msisdn|
+-------------+
|2250799875191|
|2250700606668|
|2250799896707|
|2250704679633|
|2250799870380|
|2250704679604|
|2250768767652|
|2250704679670|
|2250700180722|
|2250700808351|
|2250700697832|
|2250704655763|
|2250709949330|
|2250700691307|
|2250789378392|
|2250700316501|
|2250704695807|
|2250704697509|
|2250798949382|
|2250799338161|
|2250700437245|
|2250700437245|
|2250797264316|
|2250799896547|
|2250799896690|
|2250788074827|
|2250704679626|
|2250787631577|
|2250799534677|
|2250700687132|
|2250704663758|
|2250799638283|
|2250700686347|
|2250700691520|
|2250789089824|
|2250799219932|
|2250704697993|
|2250700691245|
|2250704679673|
|2250799896582|
|2250769728792|
|2250787426494|
|2250799966259|
|2250787967095|
|2250704679609|
|2250799896544|
|2250777514903|
|2250704138187|
|2250704669873|
|2250799104575|
|2250700437234|
|2250749449756|
|2250759128166|
|2250797258816|
|2250700691539|
|2250799668446|
|2250769924793|
|2250798825652|
|2250768715115|
|2250769

{'NOMBRE DE DETECTION': 7977170,
 'NOMBRE DE FRA': 260,
 'NOMBRE COMMUN': 115,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [30]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211010.parquet", "20211010")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|4851784|
|       1.0|1358908|
+----------+-------+



{'NOMBRE DE DETECTION': 6210692,
 'NOMBRE DE FRA': 118,
 'NOMBRE COMMUN': 14,
 'NOMBRE FRAU/NON-FRAU': None}

In [36]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_rf_20211006.parquet", "20211006")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|3485951|
|       1.0|3439878|
+----------+-------+



{'NOMBRE DE DETECTION': 6925829,
 'NOMBRE DE FRA': 364,
 'NOMBRE COMMUN': 259,
 'NOMBRE FRAU/NON-FRAU': None}

In [33]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211006.parquet", "20211006")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|5464764|
|       1.0|1461065|
+----------+-------+



{'NOMBRE DE DETECTION': 6925829,
 'NOMBRE DE FRA': 364,
 'NOMBRE COMMUN': 176,
 'NOMBRE FRAU/NON-FRAU': None}

In [34]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211007.parquet", "20211007")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|4817362|
|       1.0|1277979|
+----------+-------+



{'NOMBRE DE DETECTION': 6095341,
 'NOMBRE DE FRA': 359,
 'NOMBRE COMMUN': 201,
 'NOMBRE FRAU/NON-FRAU': None}

In [35]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211008.parquet", "20211008")

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|6186784|
|       1.0|1790386|
+----------+-------+



{'NOMBRE DE DETECTION': 7977170,
 'NOMBRE DE FRA': 260,
 'NOMBRE COMMUN': 115,
 'NOMBRE FRAU/NON-FRAU': None}

**JOURNEE DU 07**
MODELE RANDOM FOREST :
     - NOMBRE DE FRA : 359
     - NOMBRE COMMUN RF-FRA : 270
     - NOMBRE DE DETECTION RF : 3070523

MODELE ARBRE DE DECISION :
     - NOMBRE DE FRA : 
     - NOMBRE COMMUN AD-FRA : 
     - NOMBRE DE DETECTION AD :
          
JOINTURE RF ET DECISION TREE :
     - NOMBRE DE FRA: 359
     - NOMBRE COMMUN RF-AD: 200
     - NOMBRE DE DETECTION JOINTURE INNER RF-AD : 848520
     
**JOURNEE DU 08**
MODELE RANDOM FOREST :
     - NOMBRE DE FRA : 260
     - NOMBRE COMMUN RF-FRA : 141
     - NOMBRE DE DETECTION RF : 3816761
     
MODELE ARBRE DE DECISION :
     - NOMBRE DE FRA: 
     - NOMBRE COMMUN AD-FRA: 
     - NOMBRE DE DETECTION AD : 
     
JOINTURE RF ET DECISION TREE :
     - NOMBRE DE FRA: 260
     - NOMBRE COMMUN RF-AD : 141
     - NOMBRE DE DETECTION JOINTURE INNER RF-AD : 1341246
     
**JOURNEE DU 10**
MODELE RANDOM FOREST :
     - NOMBRE DE FRA: 
     - NOMBRE COMMUN RF-FRA: 
     - NOMBRE DE DETECTION RF : 
     
MODELE ARBRE DE DECISION :
     - NOMBRE DE FRA: 118
     - NOMBRE COMMUN AD-FRA : 14
     - NOMBRE DE DETECTION AD :  1358908   
     
JOINTURE RF ET DECISION TREE :
     - NOMBRE DE FRA: 118
     - NOMBRE COMMUN RF-AD: 13
     - NOMBRE DE DETECTION JOINTURE INNER RF-AD : 943755


In [14]:
prediction_rf_20211007 = spark.read.parquet("/Users/youssouf/Downloads/prediction_rf_20211007.parquet")
prediction_rf_20211007.show(5)


prediction_rf_20211007 = prediction_rf_20211007.withColumnRenamed("prediction", "prediction_rf")
prediction_rf_20211007 = prediction_rf_20211007.filter(col("prediction_rf") == 1)


+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|            features|       rawPrediction|         probability|prediction|
+-------------+---

In [15]:
# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(5).setSeed(1)
model = bkm.fit(prediction_rf_20211007)

# Make predictions
predictions = model.transform(prediction_rf_20211007)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.7118524774262149
Cluster Centers: 
[3.70105438e+00 9.76792318e-01 3.60076036e+00 4.92153305e+02
 2.01611806e+00 2.24608315e+00 7.67412720e+00 2.47020694e+00
 3.98878049e+00 5.51995841e+02 2.33141150e+00 1.13201257e-01
 5.50843418e-02 1.44409933e-02 3.39698342e+00 8.91710316e-03
 6.53013475e-01 3.60066088e-01 3.64177620e-01 7.35202660e-01
 3.14921550e-01 5.49044956e-01 2.05087412e+03 1.07391467e+03
 3.48173770e+01 9.17993296e+01 3.30730903e+00]
[8.16079650e+00 1.79316733e+00 6.50770438e+00 9.26413790e+02
 3.74058469e+00 4.96360381e+00 1.27498479e+01 3.50975404e+00
 6.29067525e+00 8.41228266e+02 3.74927761e+00 2.30198122e-01
 8.68704116e-02 2.49035657e-02 6.52691521e+00 1.58269850e-02
 9.77851208e-01 7.92959947e-01 6.72016065e-01 9.37013508e-01
 9.86412780e-01 7.57721661e-01 8.30977275e+03 2.29506940e+04
 8.16239268e+01 1.85583535e+02 4.20845373e+00]
[1.10033567e+01 2.13109111e+00 7.59899934e+00 1.04637073e+03
 4.39852250e+00 6.03195916e+00 

In [16]:
check_fraudeur_date3(predictions, 0 ,"20211007")

+----------+-------+
|prediction|  count|
+----------+-------+
|         1| 289316|
|         3| 456810|
|         4| 273770|
|         2| 361962|
|         0|1688665|
+----------+-------+



{'NOMBRE DE DETECTION': 3070523,
 'NOMBRE DE FRA': 462,
 'NOMBRE COMMUN': 270,
 'NOMBRE FRAU/NON-FRAU': None}

In [17]:
prediction_final_20211007 = spark.read.parquet("/Users/youssouf/Downloads/prediction_final_20211007.parquet")
prediction_final_20211007.show(5)


+-------------+--------------------+-------------+--------------------+-------------+
|       msisdn|      probability_dt|prediction_dt|      probability_rf|prediction_rf|
+-------------+--------------------+-------------+--------------------+-------------+
|2250700001496|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
|2250700005501|[0.08443540183112...|          1.0|[0.39126643465714...|          1.0|
|2250700009946|[0.08443540183112...|          1.0|[0.36876257793014...|          1.0|
|2250700012258|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
|2250700014334|[0.23142144638403...|          1.0|[0.29755085106931...|          1.0|
+-------------+--------------------+-------------+--------------------+-------------+
only showing top 5 rows



In [18]:
prediction_final_20211007_ = prediction_final_20211007.select("msisdn").join(prediction_rf_20211007, ["msisdn"], "inner")
prediction_final_20211007_.show(5)


+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-----------------+--------------------+--------------------+--------------------+-------------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|            features|       rawPrediction|         probability|prediction_rf|
+-----------

In [19]:
# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(5).setSeed(1)
model = bkm.fit(prediction_final_20211007_)

# Make predictions
predictions = model.transform(prediction_final_20211007_)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.8277359672120382
Cluster Centers: 
[4.29592530e-01 1.34387050e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.18112774e+00 1.37233728e+00
 6.40804121e-01 9.70428589e+01 3.60241709e-01 6.64066275e-02
 3.91703175e-02 5.19308915e-04 1.32432429e-01 3.49090993e-04
 2.94570770e-01 1.08081168e-01 1.22610277e-01 5.22515648e-01
 1.64910874e-01 3.97675227e-01 6.68116066e+02 3.84785058e+02
 7.07595614e+00 1.69084689e+01 2.27087730e+00]
[5.04973876e-01 1.57453376e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.67992414e+00 1.53474275e+00
 6.84256522e-02 9.83187371e+00 4.14458638e-02 8.72055187e-02
 4.81582715e-02 1.78501701e-03 7.62853052e-01 1.09704171e-03
 8.94274930e-01 5.11128465e-01 3.93057028e-01 9.07885685e-01
 9.61640728e-01 6.17095257e-01 7.92529103e+03 1.57899489e+04
 3.17151968e+01 6.07170004e+01 4.07768543e+00]
[7.73411562e-01 1.89400460e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 

In [42]:
check_fraudeur_date3(predictions, 0 ,"20211007")

NameError: name 'predictions' is not defined

In [50]:
# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(prediction_rf_20211007)

ll = model.logLikelihood(prediction_rf_20211007)
lp = model.logPerplexity(prediction_rf_20211007)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

KeyboardInterrupt: 

In [ ]:
check_fraudeur_date3(transformed, 3 ,"20211007")

In [ ]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211010.parquet", "20211010")

In [16]:
def kmeans_prediction(path_final, path_dt, date) :
    
    from pyspark.sql.functions import col    
    path_final = "/Users/youssouf/Downloads/" + path_final
    df_final = spark.read.parquet(path_final)
    path_dt = "/Users/youssouf/Downloads/" + path_dt
    df_dt = spark.read.parquet(path_dt)
    df_dt = df_dt.withColumnRenamed("prediction", "prediction_dt")
    
    dataset = df_final.select("msisdn").join(df_dt, ["msisdn"] ,"inner")
    
    # Trains a bisecting k-means model.
    bkm = BisectingKMeans().setK(5).setSeed(1)
    model = bkm.fit(dataset)

    # Make predictions
    predictions = model.transform(dataset)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Silhouette with squared euclidean distance = " + str(silhouette))

    # Shows the result.
    print("Cluster Centers: ")
    centers = model.clusterCenters()
    #for center in centers:
     #   print(center)
       
    path_predictions = "/Users/youssouf/Documents/simboxv2/predictions_" + str(date) + ".parquet"
    predictions = predictions.filter(col("prediction") == 0)
    predictions.write.mode("overwrite").format("parquet").save(path_predictions)
    
    dico = {str(date) : centers}
    return(dico)

In [17]:
kmeans_prediction("prediction_final_20211010.parquet", "prediction_dt_20211010.parquet", "20211010")

Silhouette with squared euclidean distance = 0.8063133568068535
Cluster Centers: 
[4.07446202e-01 1.21562752e-01 1.00618870e-02 1.44579370e+00
 6.11021116e-03 7.23887475e-03 3.45177128e+00 1.31355465e+00
 6.28156773e-01 9.73258346e+01 3.48912680e-01 6.00086341e-02
 3.49100791e-02 9.02660209e-04 1.62382752e-01 5.58918540e-04
 3.17802706e-01 1.22907946e-01 1.29349719e-01 5.70534031e-01
 1.89879379e-01 4.42545745e-01 6.43522041e+02 4.66188541e+02
 7.56345755e+00 1.70172223e+01 2.51317248e+00 3.71905479e-01
 1.72946719e-01 1.28409166e-01 4.79885022e-03 9.38780150e-02
 8.92375025e-02 4.59991312e-03 7.13697023e-02 4.57420016e-03
 4.01663493e-03 2.56317606e-03 3.43065012e-03 4.46688238e-02
 2.43190464e-03 1.16790969e-03]
[4.95656405e-01 1.38093398e-01 2.26983241e-02 3.97554134e+00
 1.48984586e-02 1.91204042e-02 3.14834056e+00 1.50252601e+00
 5.96797046e-02 8.35580267e+00 3.80547565e-02 8.50256895e-02
 3.60940564e-02 1.70308989e-03 3.73262920e-01 1.18786942e-03
 9.00963176e-01 5.27170724e-01 4

In [ ]:
kmeans_prediction("prediction_final_20211010.parquet", "prediction_dt_20211010.parquet", "20211010")

In [24]:
df = spark.read.parquet("/Users/youssouf/Documents/simboxv2/predictions_20211013.parquet")
df.show()

+-------------+----------+------------+-----------------+-------------+-------+------------------+-------------+-----------+----------------+------------+------+-----------------+--------------+-------------------+---------------+---------+--------------------+------------+-----------+----------+---------+----------+-----------+---------+------+------------+-------------------+-----------------+-----------------+----------------+--------------------+--------------+--------------------+-------------+----------+
|       msisdn|date_appel|nbre_sms_out|nbre_dist_sms_out|nbre_voix_out|dur_out|nbre_dist_voix_out|nbre_cell_out|nbre_sms_in|nbre_dist_sms_in|nbre_voix_in|dur_in|nbre_dist_voix_in|nbre_sms_night|nbre_dist_sms_night|nbre_voix_night|dur_night|nbre_dist_voix_night|distributeur|statut_voix|statut_sms|statut_ff|statut_sva|statut_data|statut_om|valeur|vol_tot_data|vol_tot_voix_offnet|vol_tot_voix_onet|distributeurIndex|distributeur_vec|            features| rawPrediction|         probab

In [25]:
df.count()

430262

#### 3 times kmeans to decrease the FP

In [ ]:
def kmeans_prediction2(path_final, path_dt, date):
    from pyspark.sql.functions import col
    path_final = "/Users/youssouf/Downloads/" + path_final
    df_final = spark.read.parquet(path_final)
    path_dt = "/Users/youssouf/Downloads/" + path_dt
    df_dt = spark.read.parquet(path_dt)
    df_dt = df_dt.withColumnRenamed("prediction", "prediction_dt")

    dataset = df_final.select("msisdn").join(df_dt, ["msisdn"], "inner")

    print("**********  APPLING  bisecting 5-means", date)
    # Trains a bisecting k-means model.
    bkm = BisectingKMeans().setK(5).setSeed(1)
    model = bkm.fit(dataset)

    # Make predictions
    predictions = model.transform(dataset)
    predictions.groupBy("predcition").count().show()
    
    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("Silhouette with squared euclidean distance = " + str(silhouette))

    # Shows the result.
    print("Cluster Centers: ")
    centers = model.clusterCenters()
    # for center in centers:
    #   print(center)
    
    
    print("Second Kmeans")
    predictions1 = predictions.withColumnRenamed("prediction", "prediction_km1")
    model = bkm.fit(predictions1.filter(col("prediction_km1") == 0))

    # Make predictions
    predictions1 = model.transform(predictions1)
    predictions1.groupBy("predcition").count().show()

    # Evaluate clustering by computing Silhouette score
    evaluator1 = ClusteringEvaluator()
    
    silhouette1 = evaluator.evaluate(predictions1)
    
    
    print("Third Kmeans")
    predictions2 = predictions1.withColumnRenamed("prediction", "prediction_km2")
    model = bkm.fit(predictions2.filter(col("prediction_km2") == 0))

    # Make predictions
    predictions2 = model.transform(predictions2)
    predictions2.groupBy("predcition").count().show()

    # Evaluate clustering by computing Silhouette score
    evaluator2 = ClusteringEvaluator()
    
    silhouette2 = evaluator.evaluate(predictions2)
    
    
    #print("**********  SAVING PREDICTIONS ", date)
    #path_predictions = "/Users/youssouf/Documents/simboxv2/predictions_" + str(date) + ".parquet"
    #predictions = predictions.filter(col("prediction") == 0)
    #predictions.write.mode("overwrite").format("parquet").save(path_predictions)

    dico = {str(date): centers, "silhouette" : silhouette, "silhouette1", silhouette1, "silhouette2" : silhouette2}
    return (dico)

In [5]:
path_final = "/Users/youssouf/Downloads/" + "prediction_final_20211010.parquet"
df_final = spark.read.parquet(path_final)
path_dt = "/Users/youssouf/Downloads/" + "prediction_dt_20211010.parquet"
df_dt = spark.read.parquet(path_dt)
df_dt = df_dt.withColumnRenamed("prediction", "prediction_dt")

dataset = df_final.select("msisdn").join(df_dt, ["msisdn"], "inner")


#dataset = spark.read.parquet("/Users/youssouf/Downloads/prediction_final_20211010.parquet")

#print("**********  APPLING  bisecting 5-means", date)
# Trains a bisecting k-means model.
bkm = BisectingKMeans(distanceMeasure='cosine').setK(5).setSeed(1)
model = bkm.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
# for center in centers:
#   print(center)

Silhouette with squared euclidean distance = -0.4215063793601907
Cluster Centers: 


In [6]:
predictions.groupBy("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|         1|188309|
|         3|113723|
|         4|169661|
|         2|192915|
|         0|296787|
+----------+------+



In [9]:
check_fraudeur_date3(predictions, 1, "20211010")

+------+
|msisdn|
+------+
+------+

+----------+------+
|prediction| count|
+----------+------+
|         1|188309|
|         3|113723|
|         4|169661|
|         2|192915|
|         0|296787|
+----------+------+



{'NOMBRE DE DETECTION': 961395,
 'NOMBRE DE FRA': 13,
 'NOMBRE COMMUN': 0,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [10]:
check_fraudeur_date3(predictions, 2, "20211010")

+-------------+
|       msisdn|
+-------------+
|2250700813425|
+-------------+

+----------+------+
|prediction| count|
+----------+------+
|         1|188309|
|         3|113723|
|         4|169661|
|         2|192915|
|         0|296787|
+----------+------+



{'NOMBRE DE DETECTION': 961395,
 'NOMBRE DE FRA': 13,
 'NOMBRE COMMUN': 1,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [11]:
check_fraudeur_date3(predictions, 3, "20211010")

+-------------+
|       msisdn|
+-------------+
|2250704679615|
|2250704679616|
|2250700606278|
|2250704679618|
|2250704679608|
|2250704679601|
|2250704679602|
|2250700147579|
|2250700606272|
|2250704679611|
|2250704680459|
|2250704679795|
+-------------+

+----------+------+
|prediction| count|
+----------+------+
|         1|188309|
|         3|113723|
|         4|169661|
|         2|192915|
|         0|296787|
+----------+------+



{'NOMBRE DE DETECTION': 961395,
 'NOMBRE DE FRA': 13,
 'NOMBRE COMMUN': 12,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [12]:
check_fraudeur_date3(predictions, 4, "20211010")

+------+
|msisdn|
+------+
+------+

+----------+------+
|prediction| count|
+----------+------+
|         1|188309|
|         3|113723|
|         4|169661|
|         2|192915|
|         0|296787|
+----------+------+



{'NOMBRE DE DETECTION': 961395,
 'NOMBRE DE FRA': 13,
 'NOMBRE COMMUN': 0,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [24]:
path_final = "/Users/youssouf/Downloads/" + "prediction_final_20211008.parquet"
df_final = spark.read.parquet(path_final)
path_dt = "/Users/youssouf/Downloads/" + "prediction_dt_20211008.parquet"
df_dt = spark.read.parquet(path_dt)
df_dt = df_dt.withColumnRenamed("prediction", "prediction_dt")

dataset = df_final.select("msisdn").join(df_dt, ["msisdn"], "inner")


#dataset = spark.read.parquet("/Users/youssouf/Downloads/prediction_final_20211010.parquet")

#print("**********  APPLING  bisecting 5-means", date)
# Trains a bisecting k-means model.
bkm = BisectingKMeans(distanceMeasure='cosine').setK(2).setSeed(12345)
model = bkm.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()

Silhouette with squared euclidean distance = 0.13199123183756317
Cluster Centers: 


In [25]:
predictions.groupBy("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|         1|687965|
|         0|663519|
+----------+------+



In [23]:
check_fraudeur_date3(predictions, 1 , "20211008")

+-------------+
|       msisdn|
+-------------+
|2250799875191|
|2250700606668|
|2250799896707|
|2250704679633|
|2250799870380|
|2250704679604|
|2250768767652|
|2250704679670|
|2250700180722|
|2250700808351|
|2250704655763|
|2250700691307|
|2250789378392|
|2250700316501|
|2250704695807|
|2250704697509|
|2250798949382|
|2250799338161|
|2250700437245|
|2250700437245|
|2250797264316|
|2250799896547|
|2250799896690|
|2250704679626|
|2250700687132|
|2250704663758|
|2250700686347|
|2250700691520|
|2250799219932|
|2250704697993|
|2250700691245|
|2250704679673|
|2250799896582|
|2250769728792|
|2250799966259|
|2250787967095|
|2250704679609|
|2250799896544|
|2250704138187|
|2250704669873|
|2250799104575|
|2250700437234|
|2250759128166|
|2250700691539|
|2250799668446|
|2250798825652|
|2250768715115|
|2250704663757|
|2250799126582|
|2250799896545|
|2250799896701|
|2250799896726|
|2250700691692|
|2250799575408|
|2250700690255|
|2250704679642|
|2250704663763|
|2250704675652|
|2250704664318|
|2250799

{'NOMBRE DE DETECTION': 1351484,
 'NOMBRE DE FRA': 101,
 'NOMBRE COMMUN': 94,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [15]:
check_fraudeur_date3(predictions, 3, "20211008")

+-------------+
|       msisdn|
+-------------+
|2250700606668|
|2250799896707|
|2250704679633|
|2250799870380|
|2250704679604|
|2250704679670|
|2250700808351|
|2250704655763|
|2250700691307|
|2250704695807|
|2250798949382|
|2250799896547|
|2250799896690|
|2250704679626|
|2250700687132|
|2250700686347|
|2250700691520|
|2250704679673|
|2250799896582|
|2250704679609|
|2250704138187|
|2250700691539|
|2250799668446|
|2250704663757|
|2250700691692|
|2250700690255|
|2250704679642|
|2250704675652|
|2250700690163|
|2250704695933|
|2250700808388|
|2250787304474|
|2250704679671|
|2250799896548|
|2250704695783|
|2250704695907|
|2250704679668|
|2250704679624|
|2250704695797|
|2250704679675|
|2250704679674|
|2250704663760|
|2250799896719|
|2250704664392|
|2250700690640|
|2250704695824|
|2250704675879|
|2250704655715|
+-------------+

+----------+------+
|prediction| count|
+----------+------+
|         1|313180|
|         3|176600|
|         4|266766|
|         2|244599|
|         0|350339|
+------

{'NOMBRE DE DETECTION': 1351484,
 'NOMBRE DE FRA': 101,
 'NOMBRE COMMUN': 48,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [19]:
check_fraudeur_date2("/Users/youssouf/Downloads/prediction_dt_20211008.parquet", "20211008")

+-------------+
|       msisdn|
+-------------+
|2250799875191|
|2250700606668|
|2250799896707|
|2250704679633|
|2250799870380|
|2250704679604|
|2250768767652|
|2250704679670|
|2250700180722|
|2250700808351|
|2250700697832|
|2250704655763|
|2250709949330|
|2250700691307|
|2250789378392|
|2250700316501|
|2250704695807|
|2250704697509|
|2250798949382|
|2250799338161|
|2250700437245|
|2250700437245|
|2250797264316|
|2250799896547|
|2250799896690|
|2250788074827|
|2250704679626|
|2250787631577|
|2250799534677|
|2250700687132|
|2250704663758|
|2250799638283|
|2250700686347|
|2250700691520|
|2250789089824|
|2250799219932|
|2250704697993|
|2250700691245|
|2250704679673|
|2250799896582|
|2250769728792|
|2250787426494|
|2250799966259|
|2250787967095|
|2250704679609|
|2250799896544|
|2250777514903|
|2250704138187|
|2250704669873|
|2250799104575|
|2250700437234|
|2250749449756|
|2250759128166|
|2250797258816|
|2250700691539|
|2250799668446|
|2250769924793|
|2250798825652|
|2250768715115|
|2250769

{'NOMBRE DE DETECTION': 7977170,
 'NOMBRE DE FRA': 260,
 'NOMBRE COMMUN': 115,
 'NUMERO COMMUN': None,
 'NOMBRE FRAU/NON-FRAU': None}

In [104]:
fr = spark.read.option("header", "true").option("delimiter", ";").csv("/Users/youssouf/Downloads/Détections SIMBOX.csv")

In [105]:
fr.show()

+--------------+---------+----------+-----+---------+-----------+
|Date_DETECTION|MSISDN_SB|NBRE_DETEC|OUTIL|OPERATEUR|DESTINATION|
+--------------+---------+----------+-----+---------+-----------+
|    14/10/2021| 00004290|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00004292|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019934|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019944|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019965|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00022943|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00023044|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00023049|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00028910|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00031097|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00045557|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00158769|         1|   FR|   ORANGE|     ON NET|
|    14/10

In [107]:
fr = fr.withColumn("msisdn", concat(lit("22507"), fr.MSISDN_SB))
fr.show()

+--------------+---------+----------+-----+---------+-----------+-------------+
|Date_DETECTION|MSISDN_SB|NBRE_DETEC|OUTIL|OPERATEUR|DESTINATION|       msisdn|
+--------------+---------+----------+-----+---------+-----------+-------------+
|    14/10/2021| 00004290|         1|   FR|   ORANGE|     ON NET|2250700004290|
|    14/10/2021| 00004292|         1|   FR|   ORANGE|     ON NET|2250700004292|
|    14/10/2021| 00019934|         1|   FR|   ORANGE|     ON NET|2250700019934|
|    14/10/2021| 00019944|         1|   FR|   ORANGE|     ON NET|2250700019944|
|    14/10/2021| 00019965|         1|   FR|   ORANGE|     ON NET|2250700019965|
|    14/10/2021| 00022943|         1|   FR|   ORANGE|     ON NET|2250700022943|
|    14/10/2021| 00023044|         1|   FR|   ORANGE|     ON NET|2250700023044|
|    14/10/2021| 00023049|         1|   FR|   ORANGE|     ON NET|2250700023049|
|    14/10/2021| 00028910|         1|   FR|   ORANGE|     ON NET|2250700028910|
|    14/10/2021| 00031097|         1|   

In [116]:
fr_21 = fr.filter(col("Date_DETECTION") == "21/10/2021")

In [117]:
predictions.filter(col("prediction") == 0).join(fr_21, ["msisdn"], "inner").select("msisdn").distinct().count()

172

In [118]:
predictions.filter(col("prediction") == 0).join(fr_21, ["msisdn"], "inner").select("msisdn").count()

178

In [125]:
fr_21.select("msisdn").distinct().count()

342

In [126]:
172/342

0.5029239766081871

In [ ]:
print("Second Kmeans")
predictions1 = predictions.withColumnRenamed("prediction", "prediction_km1")
model = bkm.fit(predictions1.filter(col("prediction_km1") == 0))

# Make predictions
predictions1 = model.transform(predictions1)

# Evaluate clustering by computing Silhouette score
evaluator1 = ClusteringEvaluator()

silhouette1 = evaluator.evaluate(predictions1)

In [ ]:
predictions1.groupBy("prediction").count().show()

In [ ]:
print("Third Kmeans")
predictions2 = predictions1.withColumnRenamed("prediction", "prediction_km2")
model = bkm.fit(predictions2.filter(col("prediction_km2") == 0))

# Make predictions
predictions2 = model.transform(predictions2)

# Evaluate clustering by computing Silhouette score
evaluator2 = ClusteringEvaluator()

silhouette2 = evaluator.evaluate(predictions2)

In [ ]:
predictions2.groupBy("prediction").count().show()

In [74]:
df_recap = pd.read_excel('/Users/youssouf/Downloads/recapitulatif_SSvsOutilsSB.xlsx',sheet_name="Detection_outils")
df_recap["MSISDN_SB"] = df_recap["MSISDN_SB"].apply(str)

df_recap.head()




,Date_DETECTION,MSISDN_SB,OUTIL,OPERATEUR,DESTINATION
0,2021-10-06,21154,FR,ORANGE,ON NET
1,2021-10-06,28907,FR,ORANGE,ON NET
2,2021-10-06,30908,FR,ORANGE,ON NET
3,2021-10-06,60144,FR,ORANGE,ON NET
4,2021-10-06,67843,FR,ORANGE,ON NET


In [103]:
df_recap.to_csv("/Users/youssouf/Downloads/recapitulatif_SSvsOutilsSB.csv", index = False, sep = ";")

In [87]:
df_recap[df_recap["Date_DETECTION"] == "2021-10-12"].shape

(328, 5)

In [88]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df_recap[df_recap["Date_DETECTION"] == "2021-10-12"].head(309)

,Date_DETECTION,MSISDN_SB,OUTIL,OPERATEUR,DESTINATION
2050,2021-10-12,13464,FR,ORANGE,ON NET
2051,2021-10-12,31098,FR,ORANGE,ON NET
2052,2021-10-12,202131,FR,ORANGE,ON NET
2053,2021-10-12,202155,FR,ORANGE,ON NET
2054,2021-10-12,203728,FR,ORANGE,ON NET
2055,2021-10-12,203740,FR,ORANGE,ON NET
2056,2021-10-12,203767,FR,ORANGE,ON NET
2057,2021-10-12,203844,FR,ORANGE,ON NET
2058,2021-10-12,203890,FR,ORANGE,ON NET
2059,2021-10-12,203891,FR,ORANGE,ON NET


In [89]:
df_recap[(df_recap["Date_DETECTION"] == "2021-10-12") & (df_recap["MSISDN_SB"].str.len() <= 7)].shape

(172, 5)

In [86]:
183/309

0.5922330097087378

In [90]:
172/328

0.524390243902439

In [91]:
df_vbm = spark.read\
    .option("header", True)\
    .option("delimiter",";")\
    .csv("/Users/youssouf/Downloads/fraudeur_simbox_vbm_all.csv")

In [92]:
df_vbm.columns

['MSISDN',
 'DISTRIBUTEUR',
 'STATUT_VOIX',
 'STATUT_SMS',
 'STATUT_FF',
 'STATUT_SVA',
 'STATUT_DATA',
 'STATUT_OM',
 'VALEUR',
 'VOL_TOT_DATA',
 'VOL_TOT_VOIX_OFFNET',
 'VOL_TOT_VOIX_ONET']

In [120]:
173 / 326

0.5306748466257669

In [94]:
df_recap = pd.read_excel('/Users/youssouf/Downloads/Détections SIMBOX.xlsx',sheet_name="Feuil1")
df_recap["MSISDN_SB"] = df_recap["MSISDN_SB"].apply(str)

In [95]:
df_recap.head()

,Date_DETECTION,MSISDN_SB,NBRE_DETEC,OUTIL,OPERATEUR,DESTINATION
0,2021-10-14,4290,1,FR,ORANGE,ON NET
1,2021-10-14,4292,1,FR,ORANGE,ON NET
2,2021-10-14,19934,1,FR,ORANGE,ON NET
3,2021-10-14,19944,1,FR,ORANGE,ON NET
4,2021-10-14,19965,1,FR,ORANGE,ON NET


In [96]:
df_recap[(df_recap["Date_DETECTION"] == "2021-10-21")].shape

(356, 6)

In [98]:
df_recap[(df_recap["Date_DETECTION"] == "2021-10-21")].head(356)

,Date_DETECTION,MSISDN_SB,NBRE_DETEC,OUTIL,OPERATEUR,DESTINATION
2702,2021-10-21,22953,1,FR,ORANGE,ON NET
2703,2021-10-21,22954,1,FR,ORANGE,ON NET
2704,2021-10-21,23060,1,FR,ORANGE,ON NET
2705,2021-10-21,57473,1,FR,ORANGE,ON NET
2706,2021-10-21,180629,1,FR,ORANGE,ON NET
2707,2021-10-21,200346,1,FR,ORANGE,ON NET
2708,2021-10-21,200403,1,FR,ORANGE,ON NET
2709,2021-10-21,202302,1,FR,ORANGE,ON NET
2710,2021-10-21,248516,1,FR,ORANGE,ON NET
2711,2021-10-21,384551,1,FR,ORANGE,ON NET


In [100]:
#df_recap[(df_recap["Date_DETECTION"] == "2021-10-21")].head(356)

In [56]:
df_fraudeur_ = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/SIMBOX1.csv")

df_fraudeur_ = df_fraudeur_.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))

df_fraudeur_ = df_fraudeur_.select("Date_DETECTION", "msisdn")

df_fraudeur = spark.read.option("header", True) \
                        .option("delimiter", ";") \
                        .csv("/Users/youssouf/Downloads/SIMBOXX.csv")

df_fraudeur = df_fraudeur.withColumn("msisdn", concat(lit("22507") , col("MSISDN_SB")))
df_fraudeur = df_fraudeur.select("Date_DETECTION", "msisdn")



In [59]:
df = df_fraudeur_.union(df_fraudeur)
df = df.withColumnRenamed("Date_DETECTION", "date_detection")

In [61]:
df.write.format("parquet").mode("overwrite").save("/Users/youssouf/Documents/simboxv2/outils_fra_detection.parquet")

In [62]:
def From_pyparquet_to_pandascsv2(parquet_path, csv_name) :
    df = pq.read_table(parquet_path).to_pandas()
    csv_save_path = "/Users/youssouf/Documents/simboxv2/" + csv_name + ".csv"
    df.to_csv(csv_save_path, index = False, sep = ";" )
    return os.path.isfile(csv_save_path)

In [63]:
From_pyparquet_to_pandascsv2("/Users/youssouf/Documents/simboxv2/outils_fra_detection.parquet", "outils_fra_detection")

True

In [52]:
df_fraudeur.show(5)

+--------------+---------+----------+-----+---------+-----------+
|Date_DETECTION|MSISDN_SB|NBRE_DETEC|OUTIL|OPERATEUR|DESTINATION|
+--------------+---------+----------+-----+---------+-----------+
|    14/10/2021| 00004290|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00004292|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019934|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019944|         1|   FR|   ORANGE|     ON NET|
|    14/10/2021| 00019965|         1|   FR|   ORANGE|     ON NET|
+--------------+---------+----------+-----+---------+-----------+
only showing top 5 rows



In [53]:
df_fraudeur_.show(5)

+--------------+---------+-----+---------+-----------+
|Date_DETECTION|MSISDN_SB|OUTIL|OPERATEUR|DESTINATION|
+--------------+---------+-----+---------+-----------+
|    28/09/2021| 00004846|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00048563|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058736|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058747|   FR|   ORANGE|     ON NET|
|    28/09/2021| 00058748|   FR|   ORANGE|     ON NET|
+--------------+---------+-----+---------+-----------+
only showing top 5 rows

